In [ ]:
%matplotlib inline

# Extract geometries from mongoDB, export as SVG for nesting, stitch nested SVGs

This notebook extracts geometries (areas, like polygons of parking spaces) from a mongoDB, then exports all areas in an svg file for nesting with SVGNest. In the end, the SVG bins are stitched back together.

Created on:  2016-10-28  
Last update: 2016-11-11  
Contact: michael.szell@moovel.com, michael.szell@gmail.com (Michael Szell)

### Preliminaries

In [ ]:
from __future__ import unicode_literals
import sys
import csv
import os
import math
from random import shuffle, choice, uniform
import random
import pprint
import requests
import gzip
from collections import defaultdict
import time
import datetime
import numpy as np
from scipy import stats
import pyprind
import itertools
import logging
from ast import literal_eval as make_tuple
from collections import OrderedDict
from retrying import retry

import json
from xml.dom import minidom
from shapely.geometry import mapping, shape, LineString, LinearRing, Polygon, MultiPolygon
import shapely
import shapely.ops as ops
from functools import partial
import pyproj
RobinsonProjection = pyproj.Proj("+proj=robin +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs")
from scipy import spatial
from haversine import haversine
import overpass
apiop = overpass.API(timeout=600)

import pymongo
from pymongo import MongoClient

# plotting stuff
import matplotlib.pyplot as plt
# import mpld3
# mpld3.enable_notebook() # unfortunately, this is too buggy: https://github.com/mpld3/mpld3/issues/193
# mpld3.disable_notebook()

curtime = time.strftime("%Y%m%d_%H%M%S")
pp = pprint.PrettyPrinter(indent=4)

# City parameters (generalize later)
cityname = "vienna" # "amsterdam" b"berlin"

### Functions

In [ ]:
def coordinatesToSVGString(coo, xoffset = 0, yoffset = 0, idname = ""):
    svgstring = "\n  <polygon "
    if idname:
        svgstring += "class=\""+idname+"\""
    svgstring += " points=\""
    strxylist = [str(coo[i][0]+xoffset)+","+str(coo[i][1]+yoffset) for i in range(coo.shape[0])]
    for s in strxylist:
        svgstring += s+" "
    svgstring += "\"/>"
    return svgstring

def drawPolygon(poly, title=""): # poly is a shapely Polygon
    x, y = poly.exterior.xy
    fig = plt.figure(figsize=(4,4), dpi=90)
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.plot(x, y)
    
def getLargestSubPolygon(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = 0
        j = 0
        for i in range(len(multipoly)):
            if multipoly[i].area > a:
                j = i
                a = multipoly[i].area
        return multipoly[j]
    
def getSmallestSubPolygon(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = float("inf")
        j = 0
        for i in range(len(multipoly)):
            if multipoly[i].area < a:
                j = i
                a = multipoly[i].area
        return multipoly[j]

def getTwoLargestSubPolygons(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give two largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = [multipoly[i].area for i in range(len(multipoly))]
        sortorder = sorted(range(len(a)), key=lambda k: a[k], reverse=True) # http://stackoverflow.com/questions/7851077/how-to-return-index-of-a-sorted-list
        return MultiPolygon([ multipoly[i] for i in sortorder[0:2] ])
    

In [ ]:
# mongo connection
client = MongoClient()
db = client[cityname+'_derived']
ways = db['ways']
cursor = ways.find({"$and": [{"properties.amenity": "parking"}, {"geometry.type": "Polygon"}]})
numparkingareas = cursor.count()
print(numparkingareas)

## Get parking spaces for multiple SVG bins

### Features
1. add buffer around big tiles ✔️
2. give more bin space for small tiles on top of bins ✔️
3. pre-select tiles from DB and exhaust them fully (and uniquely!)
4. do not order big tiles by size, but more randomly ✔️
5. add id as class ✔️
6. prevent shadows

In [16]:
pathdatain = '/Users/szellmi/Documents/lab-mobviz/_playground/parking-nesting/output/'+cityname+'in/'
pathdataout = '/Users/szellmi/Documents/lab-mobviz/_playground/parking-nesting/output/'+cityname+'out/'
parameters = {"berlin":{"maxbigparts":50, "maxbins":7}, "newyork":{"maxbigparts":50, "maxbins":5}, "stuttgart":{"maxbigparts":30, "maxbins":2}}

In [ ]:
# get parking spaces (ALL in a city)
random.seed(0)
maxbigparts = 30
scale = 0.6
binareafactor = 0.8 #1.75
maxbins = 4
smallvsmedium = 11
buffereps = 5
height = 800 # 1131
width = 800 - 1.5*buffereps
draw = False
eps = 0.000001

bigbin = Polygon([[0,0], [width,0], [width, maxbins*height], [0, maxbins*height]])
bigbin = bigbin.difference(Polygon([[width/2-eps,-1], [width/2-eps,maxbins*height-2], [width/2+eps,maxbins*height-2], [width/2+eps,-1]]))

# pre-select all parts
idsused = set()
idsnotused = set()
indicesused = set()
alltiles = []
for i,way in enumerate(ways.find({"$and": [{"properties.amenity": "parking"}, {"geometry.type": "Polygon"}, {"properties_derived.area": { "$gte": 12 }}]}).sort("properties_derived.area",-1)):
    npway = np.asarray(way["geometry"]["coordinates"])
    npwayxy = [RobinsonProjection(npway[i][0], npway[i][1]) for i in range(npway.shape[0])]
    npwayxy = np.asarray([[npwayxy[i][0],-npwayxy[i][1]] for i in range(npway.shape[0])])
    objectwidth = max(npwayxy[:, 0])-min(npwayxy[:, 0])
    npwayxy[:, 0] -= min(npwayxy[:, 0])
    npwayxy[:, 1] -= min(npwayxy[:, 1])
    npwayxy *= scale
    objectwidth *= scale
    if objectwidth < width:
        objectheight = max(npwayxy[:, 1])
        idsnotused.add(way["_id"])
        coo = [[npwayxy[k][0], npwayxy[k][1]] for k in range(npwayxy.shape[0])]
        area = Polygon(coo).buffer(buffereps/2).area
    #     print(str(area))
    #     print(str(Polygon(coo).area))
    #     print(str(way["properties_derived"]["area"]))
        alltiles.append( { "_id": way["_id"], "width": objectwidth, "height": objectheight, "area": area, "coordinates": coo })
    else:
        print("Object "+str(way["_id"])+" was too wide (" +str(objectwidth)+ " pixel) and was ignored.")
    
bigpartstodiff = []
partareastaken = []
ypos = -0.5*buffereps
numbigparts = 0
randomizedindices = list(range(maxbigparts))
shuffle(randomizedindices)
# Determine big parts
for i in randomizedindices:
    tile = alltiles[i]
    if ypos >= height*maxbins-1-tile["height"]: # see if this part still fits
        break
    tile["coordinates"] = [[tile["coordinates"][k][0]+width/2-tile["width"]/2, tile["coordinates"][k][1]+ypos] for k in range(np.array(tile["coordinates"]).shape[0])]
    bigpartstodiff.append( tile )
    ypos += tile["height"] + 1*buffereps
        
    indicesused.add(i)
    idsused.add(tile["_id"])
    idsnotused.remove(tile["_id"])
    numbigparts += 1 # increase number of parts in any case
    if draw:
        drawPolygon(Polygon(bigpartstodiff[-1]["coordinates"]), "Big part")

# Export the big parts
svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(height)+"px\">"
for i in range(len(bigpartstodiff)):
    svg += coordinatesToSVGString(np.array(bigpartstodiff[i]["coordinates"]), 0, 0, str(bigpartstodiff[i]["_id"]))
svg += "\n</svg>"
with open(pathdataout + "bigparts.svg", "w") as f:
    f.write(svg)
            
# Clip bins into batches and diff the big parts
for j in range(numbigparts):
    bigbin = getLargestSubPolygon(bigbin.difference(Polygon(bigpartstodiff[j]["coordinates"]).buffer(1.75*buffereps, 1, 2, 2)))
        
# Cut the big part into sub-bins
scissorv = Polygon([[width/2-eps, -1], [width/2-eps, maxbins*height+1], [width/2+eps, maxbins*height+1], [width/2+eps, -1]])
cutbins = [[], []]
bigbin = getTwoLargestSubPolygons(bigbin.difference(scissorv)) # cut in half vertically

for m in range(len(bigbin)): # cut horizontally
    rest = bigbin[m]
    for i in range(maxbins):
        scissorh = Polygon([[-1, (i+1)*height], [width+1, (i+1)*height], [width+1, (i+1)*height+eps], [-1, (i+1)*height+eps]])
        temp = rest
        temp = getTwoLargestSubPolygons(temp.difference(scissorh))
        cutbins[m].append(getSmallestSubPolygon(temp).buffer(0.75*buffereps, 1, 2, 2))
        rest = getLargestSubPolygon(temp)
        if draw:
            drawPolygon(cutbins[m][-1], "Bin")

# Fill with small parts
remainingtiles = len(idsnotused)
randomizedindices_medium = list(range(round(remainingtiles/smallvsmedium)+numbigparts))
randomizedindices_medium = list(set(randomizedindices_medium) - indicesused)
shuffle(randomizedindices_medium)
randomizedindices_small = set(list(range(remainingtiles+numbigparts))) - set(randomizedindices_medium)
randomizedindices_small = list(randomizedindices_small - indicesused)
shuffle(randomizedindices_small)

imedium = 0
ismall = 0
for numbin in range(maxbins):
    for mirrored in [0,1]:
        if mirrored:
            m = -1
        else:
            m = 1
        binarea = binareafactor*cutbins[mirrored][numbin].area
        binbound = np.array(cutbins[mirrored][numbin].exterior.coords)
        binbound = np.asarray([[-m*(binbound[i,0]-min(binbound[:,0])/2),binbound[i,1]-min(binbound[:,1])] for i in range(binbound.shape[0])])
        xpos = 0
        ypos = height+1 # start placing the elements below the bin
        yextent = 0
        svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(height)+"px\">"
        svg += coordinatesToSVGString(binbound, 0, 0, "bin")
        totalarea = 0
        
        for j in range(len(idsnotused)):
            if len(idsnotused) == 0:
                    break
            try:
                i = randomizedindices_medium[imedium]
                imedium += 1
            except: # no more medium tiles left
                i = randomizedindices_small[ismall]
                ismall += 1
            tile = alltiles[i]
            if tile["width"] <= width:
                if xpos + tile["width"] + 1 <= width: # there is space in this row
                    xdelta = m*(xpos+1)
                    ydelta = ypos
                else: # new row
                    xdelta = 0
                    ypos += yextent
                    yextent = 0
                    ydelta = ypos
                    xpos = 0
                svg += coordinatesToSVGString(np.array([[m*tile["coordinates"][k][0], tile["coordinates"][k][1]] for k in range(np.array(tile["coordinates"]).shape[0])]), xdelta, ydelta, str(tile["_id"]))
                yextent = max([yextent, tile["height"]])
                xpos += tile["width"]+1
                idsused.add(tile["_id"])
                idsnotused.remove(tile["_id"])
                totalarea += tile["area"]
                if totalarea > binarea:
                    print("Tiles area exceeded bin area - bin "+str(numbin).zfill(3)+"_"+str(mirrored)+" completed!")
                    break
            else:
                print("Object "+str(way["_id"])+" was too wide (" +str(max(npwayxy[:, 0]))+ " pixel) and could not be placed.")
            for kk in range(smallvsmedium):
                if len(idsnotused) == 0:
                    break
                try:
                    i = randomizedindices_small[ismall]
                    ismall += 1
                except:
                    i = randomizedindices_medium[imedium]
                    imedium += 1
                tile = alltiles[i]
                if tile["width"] <= width:
                    if xpos + tile["width"] + 1 <= width: # there is space in this row
                        xdelta = m*(xpos+1)
                        ydelta = ypos
                    else: # new row
                        xdelta = 0
                        ypos += yextent
                        yextent = 0
                        ydelta = ypos
                        xpos = 0
                    svg += coordinatesToSVGString(np.array([[m*tile["coordinates"][k][0], tile["coordinates"][k][1]] for k in range(np.array(tile["coordinates"]).shape[0])]), xdelta, ydelta, str(tile["_id"]))
                    yextent = max([yextent, tile["height"]])
                    xpos += tile["width"]+1
                    idsused.add(tile["_id"])
                    idsnotused.remove(tile["_id"])
                    totalarea += tile["area"]
                    if totalarea > binarea:
                        print("Tiles area exceeded bin area - bin "+str(numbin).zfill(3)+"_"+str(mirrored)+" completed!")
                        break
                else:
                    print("Object "+str(way["_id"])+" was too wide (" +str(max(npwayxy[:, 0]))+ " pixel) and could not be placed.")
            if totalarea > binarea:
                print("Tiles area exceeded bin area - bin "+str(numbin).zfill(3)+"_"+str(mirrored)+" completed!")
                break
        svg += "\n</svg>"

        if mirrored:
            with open(pathdatain + cityname + "parking"+ str(numbin).zfill(3)  +"min.svg", "w") as f:
                f.write(svg)
        else:   
            with open(pathdatain + cityname + "parking"+ str(numbin).zfill(3)  +"in.svg", "w") as f:
                f.write(svg)

## After SVGNest was executed, stich back together the parts

In [ ]:
# Collect the pieces that couldn't be fit, and fit into a last bin (TODO)


In [ ]:
def getCoordinatesFromSVG(filepath, reversexdir = False, b = 1): # The SVG needs to have polygons with classes, embedded in gs
    doc = minidom.parse(filepath)  # parseString also exists
    path_strings = [path.getAttribute('points') for path
                    in doc.getElementsByTagName('polygon')]
    class_strings = [path.getAttribute('class') for path
                    in doc.getElementsByTagName('polygon')]
    g_strings = [path.getAttribute('transform') for path
                in doc.getElementsByTagName('g')]
    doc.unlink()
    data = dict()
    numbins = 0
    for i,path in enumerate(path_strings):
        if class_strings[i] == "bin":
            numbins += 1
        if numbins == b:
            path = path.split()
            coo = []
            for temp in path:
                p = temp.split(",")
                try:
                    trans = g_strings[i] # looks like this: "translate(484.1119359029915 -1573.8819930603422) rotate(0)"
                    trans = trans.split()
                    trans = [float(trans[0][10:]), float(trans[1][0:-1])]
                except:
                    trans = [0,0]
                if reversexdir:
                    coo.append([-(float(p[0])+trans[0]), float(p[1])+trans[1]])
                else:
                    coo.append([float(p[0])+trans[0], float(p[1])+trans[1]])
    #             print(trans)
            data[class_strings[i]] = coo
        elif numbins > b:
            break
    return data

In [ ]:
# read SVG
numbins = 3#maxbins
alltiles = []

# Big parts
alltiles.append(getCoordinatesFromSVG(pathdataout + "bigparts.svg", True, 0))
ypos = 0
xpos = 0

ypos -= 0.75*buffereps
xpos = width/2

# Rest
for i in range(numbins):
    for mirrored in [0,1]:
        if mirrored:
            m = -1
            tiles = getCoordinatesFromSVG(pathdataout + str(i).zfill(3) + "m.svg", mirrored)
        else:
            m = 1
            tiles = getCoordinatesFromSVG(pathdataout + str(i).zfill(3) + ".svg", mirrored)
        for key in tiles:
            if not key == "bin":
                npwayxy = np.array(tiles[key])
                npwayxy = [[npwayxy[k,0]+xpos, npwayxy[k,1]+ypos] for k in range(npwayxy.shape[0])]
                alltiles.append({key: npwayxy})
    ypos += height

# Export
svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(height*numbins)+"px\">"
for j, tile in enumerate(alltiles):
    for i in tile:
        svg += coordinatesToSVGString(np.array([[tile[i][k][0], tile[i][k][1]] for k in range(np.array(tile[i]).shape[0])]), 0, 0, i)
svg += "\n</svg>"
with open(pathdataout + "all.svg", "w") as f:
    f.write(svg)
